In [ ]:
import boto3
import botocore


sns = boto3.client('sns')
sqs = boto3.client('sqs')

In [ ]:
import os
account_no = os.environ['ACCOUNT_NO']

### Set Up Delivery Failure Notification

In [ ]:
failureNotificationRoleArn = 'arn:aws:iam::{}:role/SNSFailureFeedback'.format(account_no)

In [ ]:
# Set this to the serverless deployment stage name
stage = 'dev'
topic_arn = 'arn:aws:sns:us-east-1:' + account_no + ':t1-' + stage

In [ ]:
# Modify the topic to enable delivery failure logging. This cannot be done via cloud
# formation at the moment.
tar = sns.set_topic_attributes(
    TopicArn=topic_arn,
    AttributeName='SQSFailureFeedbackRoleArn',
    AttributeValue=failureNotificationRoleArn
)
print(tar)

### Set Up a Doomed SQS Subscription

This is to force delivery failures so we can detect and process those.

In [ ]:
cq = sqs.create_queue(
    QueueName='sampleQueue-' + stage
)
print(cq)
queue_url = cq['QueueUrl']
queue_arn = 'arn:aws:sqs:us-east-1:' + account_no + ':sampleQueue-' + stage

In [ ]:
sns.subscribe(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':t1-' + stage,
    Protocol='sqs',
    Endpoint=queue_arn
)

In [ ]:
def allow_sns_to_write_to_sqs(topicarn, queuearn):
    policy_document = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnLike":{{
          "aws:SourceArn": "{}noworky"
        }}
      }}
    }}
  ]
}}""".format(queuearn, topicarn)

    return policy_document

In [ ]:
#policy_json = allow_sns_to_write_to_sqs(topic_arn1, queue_arn)
policy_json = allow_sns_to_write_to_sqs('arn:aws:sns:us-east-1:' + account_no + ':*', queue_arn)

response = sqs.set_queue_attributes(
    QueueUrl = queue_url,
    Attributes = {
        'Policy' : policy_json
    }
)
print(response)

### Delivery Log Queries

In [ ]:
client = boto3.client('logs')

In [ ]:
import time
nowish = int(time.time())
hour_ago = nowish - (60*60)

q = client.start_query(
    logGroupName='sns/us-east-1/427848627088/t1-' + stage + '/Failure',
    startTime=hour_ago,
    endTime=nowish,
    queryString='fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

In [ ]:
r = client.get_query_results(queryId=q['queryId'])
print(r)

### Publish Log Queries

In [ ]:
# Everything in the log
import time
nowish = int(time.time())
hour_ago = nowish - (60*60)

q = client.start_query(
    logGroupName='/aws/lambda/wrap-and-pub-dev-wrapPub',
    startTime=hour_ago,
    endTime=nowish,
    queryString='fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

In [ ]:
# Just the correlations
import time
nowish = int(time.time())
hour_ago = nowish - (15*60)

q = client.start_query(
    logGroupName='/aws/lambda/wrap-and-pub-dev-wrapPub',
    startTime=hour_ago,
    endTime=nowish,
    queryString='filter @message like /PublishContext/ | fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

In [ ]:
r = client.get_query_results(queryId=q['queryId'])
print(r)

### Clean Up

In [ ]:
try:
    sqs.delete_queue(
        QueueUrl='https://queue.amazonaws.com/' + account_no + '/sampleQueue'
    )
    print('queue deleted')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AWS.SimpleQueueService.NonExistentQueue':
        print('queue deleted')
    else:
        raise error